In [1]:
from download import download
from rope.base.pycore import perform_soa_on_changed_scopes

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:02<00:00, 4.42MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [5]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision,transforms
from mindspore.dataset import MnistDataset

In [10]:
def datapipe(dataset,batch_size):
    image_transorms=[
        vision.Rescale(1.0/255.0,0),
        vision.Normalize([0.1307],[0.3081]),
        vision.HWC2CHW()
    ]
    label_transforms=transforms.TypeCast(mindspore.int32)
    dataset=dataset.map(input_columns="image",operations=image_transorms)
    dataset=dataset.map(input_columns="label",operations=label_transforms)
    dataset=dataset.batch(batch_size,drop_remainder=True)
    return dataset
train_dataset=MnistDataset(path+"/MNIST_Data/train",num_parallel_workers=8)
test_dataset=MnistDataset(path+"/MNIST_Data/test",num_parallel_workers=8)
train_dataset=datapipe(train_dataset,64)
test_dataset=datapipe(test_dataset,64)

In [11]:
for image,label in test_dataset.create_tuple_iterator():
    print(f"shape of image [N,C,H,W]:{image.shape} {image.dtype}")
    print(f"shape of label [N]:{label.shape} {label.dtype}")
    break

shape of image [N,C,H,W]:(64, 1, 28, 28) Float32
shape of label [N]:(64,) Int32


In [14]:
for data in test_dataset.create_dict_iterator():
    print(f"shape of image [N,C,H,W]:{data['image'].shape} {data['image'].dtype}")
    print(f"shape of label [N]:{data['label'].shape} {data['label'].dtype}")
    break

shape of image [N,C,H,W]:(64, 1, 28, 28) Float32
shape of label [N]:(64,) Int32


In [19]:
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.dense_relu_sequential=nn.SequentialCell(
            nn.Dense(784,128),
            nn.ReLU(),
            nn.Dense(128,128),
            nn.ReLU(),
            nn.Dense(128,10)
        )
    def construct(self,x):
        x=self.flatten(x)
        logits=self.dense_relu_sequential(x)
        return logits

model=Network()
print(model)

Network(
  (flatten): Flatten()
  (dense_relu_sequential): SequentialCell(
    (0): Dense(input_channels=784, output_channels=128, has_bias=True)
    (1): ReLU()
    (2): Dense(input_channels=128, output_channels=128, has_bias=True)
    (3): ReLU()
    (4): Dense(input_channels=128, output_channels=10, has_bias=True)
  )
)


In [23]:
loss_fn=nn.CrossEntropyLoss()
optimizer=nn.SGD(model.trainable_params(),1e-3)

def forward_fn(data,label):
    logits=model(data)
    loss=loss_fn(logits,label)
    return loss,logits
gard_fn=mindspore.value_and_grad(forward_fn,None,optimizer.parameters,has_aux=True)

def train_step(data,label):
    (loss,_),grads=gard_fn(data,label)
    optimizer(grads)
    return loss
def train(model,dataset):
    size=dataset.get_dataset_size()
    model.set_train()
    for batch,(data,label) in enumerate(dataset.create_tuple_iterator()):
        loss=train_step(data,label)

        if batch%100==0:
            loss,current=loss.asnumpy(),batch
            print(f"loss:{loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(model,dataset,loss_fn):
    num_batches=dataset.get_dataset_size()
    model.set_train(False)
    total,test_loss,correct=0,0,0
    for data,label in dataset.create_tuple_iterator():
        pred=model(data)
        total+=len(data)
        test_loss+=loss_fn(pred,label).asnumpy()
        correct+=(pred.argmax(1)==label).asnumpy().sum()
    test_loss/=num_batches
    accuracy=correct/total
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs=3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model,train_dataset)
    test(model,test_dataset,loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.313232  [    0/  937]
loss:2.294349  [  100/  937]
loss:2.258330  [  200/  937]
loss:2.217244  [  300/  937]
loss:2.209415  [  400/  937]
loss:2.156688  [  500/  937]
loss:2.127749  [  600/  937]
loss:2.053791  [  700/  937]
loss:1.922803  [  800/  937]
loss:1.971449  [  900/  937]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.931067 

Epoch 2
-------------------------------
loss:1.898625  [    0/  937]
loss:1.824715  [  100/  937]
loss:1.856957  [  200/  937]
loss:1.802053  [  300/  937]
loss:1.691110  [  400/  937]
loss:1.603283  [  500/  937]
loss:1.433689  [  600/  937]
loss:1.275816  [  700/  937]
loss:1.299370  [  800/  937]
loss:1.263999  [  900/  937]
Test Error: 
 Accuracy: 78.2%, Avg loss: 1.168868 

Epoch 3
-------------------------------
loss:1.091936  [    0/  937]
loss:1.086840  [  100/  937]
loss:1.123651  [  200/  937]
loss:0.998243  [  300/  937]
loss:0.867913  [  400/  937]
loss:0.801303  [  500/  937]
loss:0.829653  [  600/